In [72]:
import requests 
import json
import base64
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

load_dotenv()


True

In [73]:

# Fetch Spotify API credentials from environment variables
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

#Get Spotify API token
def get_token():
    auth_str = f"{client_id}:{client_secret}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()

    headers = {
        'Authorization': f'Basic {b64_auth_str}',
    }

    data = {
        'grant_type': 'client_credentials'
    }

    response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data)
    response_data = response.json()
    return response_data['access_token']




In [74]:
access_token = get_token()

In [75]:
print(access_token)

BQAtdQt3wVOzVbVlEbBSp9JQSbl3CIO080ZnhUmRgUpC2F-R35xvKHnAI4R91SvZYm62ixtFzXdM1TFPPYHc4CkIg0n0O8q6OTeKv25JAsrLRgbowE0


In [76]:
# Function to get top 10 artists
def get_top_10_artists(access_token):
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Spotify's recommendation or browse API endpoints don't have a direct way to get top global artists,
    # so here we use a popular playlist's artists to mimic the functionality.
    # Here, we're using the 'Today's Top Hits' playlist as a proxy to get popular artists.
    playlist_id = '37i9dQZF1DXcBWIGoYBM5M'
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    response = requests.get(url, headers=headers)
    tracks = response.json()['items']

    # Extract unique artist IDs
    artist_ids = []
    for track in tracks:
        for artist in track['track']['artists']:
            if artist['id'] not in artist_ids:
                artist_ids.append(artist['id'])
    
    artist_ids = artist_ids[:50]  # Get only the top 10 unique artist IDs

    # Get artist details
    artists_url = f"https://api.spotify.com/v1/artists?ids={','.join(artist_ids)}"
    artists_response = requests.get(artists_url, headers=headers)
    artists_data = artists_response.json()['artists']

    # Collect artist information
    artists_list = []
    for artist in artists_data:
        artists_list.append({
            'ID': artist['id'],
            'Name': artist['name'],
            'Followers': artist['followers']['total'],
            'Popularity': artist['popularity'],
            'Genres': ', '.join(artist['genres'])
        })

    return artists_list

In [77]:
top_artists = get_top_10_artists(access_token)

In [78]:
print(top_artists)

[{'ID': '74KM79TiuVKeVCqs8QtB0B', 'Name': 'Sabrina Carpenter', 'Followers': 8915339, 'Popularity': 86, 'Genres': 'pop'}, {'ID': '6qqNVTkY8uBg9cP3Jd7DAH', 'Name': 'Billie Eilish', 'Followers': 95096599, 'Popularity': 94, 'Genres': 'art pop, pop'}, {'ID': '2FXC3k01G6Gw61bmprjgqS', 'Name': 'Hozier', 'Followers': 7209858, 'Popularity': 85, 'Genres': 'irish singer-songwriter, modern rock, pov: indie'}, {'ID': '246dkjvS1zLTtiykXe5h60', 'Name': 'Post Malone', 'Followers': 44114143, 'Popularity': 89, 'Genres': 'dfw rap, melodic rap, pop, rap'}, {'ID': '4oUHIQIBe0LHzYfvXNW4QM', 'Name': 'Morgan Wallen', 'Followers': 9587764, 'Popularity': 89, 'Genres': 'contemporary country'}, {'ID': '1WaFQSHVGZQJTbf0BdxdNo', 'Name': 'Tommy Richman', 'Followers': 421967, 'Popularity': 81, 'Genres': 'chill abstract hip hop'}, {'ID': '3y2cIKLjiOlp1Np37WiUdH', 'Name': 'Shaboozey', 'Followers': 369107, 'Popularity': 79, 'Genres': 'pop rap'}, {'ID': '66CXWjxzNUsdJxJ2JdwvnR', 'Name': 'Ariana Grande', 'Followers': 9680

In [79]:
#Extract data where name = Taylor Swift or  BTS
df = pd.DataFrame(top_artists)
print(df)

                        ID               Name  Followers  Popularity  \
0   74KM79TiuVKeVCqs8QtB0B  Sabrina Carpenter    8915339          86   
1   6qqNVTkY8uBg9cP3Jd7DAH      Billie Eilish   95096599          94   
2   2FXC3k01G6Gw61bmprjgqS             Hozier    7209858          85   
3   246dkjvS1zLTtiykXe5h60        Post Malone   44114143          89   
4   4oUHIQIBe0LHzYfvXNW4QM      Morgan Wallen    9587764          89   
5   1WaFQSHVGZQJTbf0BdxdNo      Tommy Richman     421967          81   
6   3y2cIKLjiOlp1Np37WiUdH          Shaboozey     369107          79   
7   66CXWjxzNUsdJxJ2JdwvnR      Ariana Grande   96806497          90   
8   06HL4z0CvFAxyc27GXpf02       Taylor Swift  112195477         100   
9   22wbnEMDvgVIAGdFeek6ET       Benson Boone    2709364          84   
10  0PCCGZ0wGLizHt2KZ7hhA2            Artemas     709215          81   
11  7dGJo4pcD2V6oG8kP0tJRR             Eminem   85098821          89   
12  2YZyLoL8N0Wb9xBt1NhZWg     Kendrick Lamar   29733236        

In [80]:
#Order by followers
df = df.sort_values(by='Followers', ascending=False)

In [81]:
print(df)

                        ID               Name  Followers  Popularity  \
8   06HL4z0CvFAxyc27GXpf02       Taylor Swift  112195477         100   
7   66CXWjxzNUsdJxJ2JdwvnR      Ariana Grande   96806497          90   
1   6qqNVTkY8uBg9cP3Jd7DAH      Billie Eilish   95096599          94   
11  7dGJo4pcD2V6oG8kP0tJRR             Eminem   85098821          89   
35  1Xyo4u8uXC1ZmMpatF05PJ         The Weeknd   84410401          92   
3   246dkjvS1zLTtiykXe5h60        Post Malone   44114143          89   
42  6M2wZ9GZgrQXHCFfjv46we           Dua Lipa   43574399          87   
30  6vWDO969PvNqNYHIOW5v0m            Beyoncé   37624279          86   
28  64KEffDW9EtZ1y2vBYgq8T         Marshmello   33631859          82   
12  2YZyLoL8N0Wb9xBt1NhZWg     Kendrick Lamar   29733236          92   
46  1Cs0zKBU1kc0i8ypK3B9ai       David Guetta   25552627          87   
40  1mcTU81TzQhprhouKaTkpq     Rauw Alejandro   21873287          87   
17  7tYKF4w9nC0nq9CsPZTHyP                SZA   20476862        

In [82]:
# Function to get an artist's data including monthly listeners (if available)
def search_spotify_artist(name, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'q': name,
        'type': 'artist',
        'limit': 1
    }
    response = requests.get('https://api.spotify.com/v1/search', headers=headers, params=params)
    if response.status_code == 200:
        artist_items = response.json()['artists']['items']
        if artist_items:
            artist = artist_items[0]
            return artist['id'], artist['name']
        else:
            print("Artist not found.")
            return None, None
    else:
        print(f"Error: {response.status_code}")
        return None, None


In [83]:
# Example usage
artist_name = 'BTS'
artist_id, artist_name = search_spotify_artist(artist_name, access_token)
if artist_id:
    print(f"Spotify Artist ID for {artist_name}: {artist_id}")

Spotify Artist ID for BTS: 3Nrfpe0tUJi4K4DXYWgMUX


In [84]:
def get_artist_data(artist_id, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [85]:
#Search for BTS data
bts_data = get_artist_data('3Nrfpe0tUJi4K4DXYWgMUX', access_token)

In [86]:
print(bts_data)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'}, 'followers': {'href': None, 'total': 73391231}, 'genres': ['k-pop', 'k-pop boy group', 'pop'], 'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX', 'id': '3Nrfpe0tUJi4K4DXYWgMUX', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebd642648235ebf3460d2d1f6a', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174d642648235ebf3460d2d1f6a', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178d642648235ebf3460d2d1f6a', 'width': 160}], 'name': 'BTS', 'popularity': 84, 'type': 'artist', 'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'}


In [94]:
def get_artist_top_tracks(artist_id, access_token, market='US'):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks'
    params = {
        'market': market  # Specify the market
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json().get('tracks', [])
    else:
        print(f"Error: {response.status_code}")
        return []

In [95]:
def get_artist_albums(artist_id, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums'
    params = {
        'market': 'US',
        'limit': 20  # You can adjust the limit
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json().get('items', [])
    else:
        print(f"Error: {response.status_code}")
        return []

In [96]:
def get_artist_playlists(artist_id, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    url = f'https://api.spotify.com/v1/search'
    params = {
        'q': f'artist:{artist_id}',
        'type': 'playlist',
        'limit': 10
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json().get('playlists', {}).get('items', [])
    else:
        print(f"Error: {response.status_code}")
        return []

In [97]:
def get_market_specific_popularity(artist_id, access_token, markets):
    popularity_data = {}
    for market in markets:
        top_tracks = get_artist_top_tracks(artist_id, access_token, market)
        total_popularity = sum(track['popularity'] for track in top_tracks)
        average_popularity = total_popularity / len(top_tracks) if top_tracks else 0
        popularity_data[market] = average_popularity
    return popularity_data

In [98]:
def compare_artists(artist_1, artist_2, access_token):
    artist_1_id, artist_1_name = search_spotify_artist(artist_1, access_token)
    artist_2_id, artist_2_name = search_spotify_artist(artist_2, access_token)

    if artist_1_id and artist_2_id:
        artist_1_data = get_artist_data(artist_1_id, access_token)
        artist_2_data = get_artist_data(artist_2_id, access_token)

        artist_1_tracks = get_artist_top_tracks(artist_1_id, access_token)
        artist_2_tracks = get_artist_top_tracks(artist_2_id, access_token)

        artist_1_albums = get_artist_albums(artist_1_id, access_token)
        artist_2_albums = get_artist_albums(artist_2_id, access_token)

        artist_1_playlists = get_artist_playlists(artist_1_id, access_token)
        artist_2_playlists = get_artist_playlists(artist_2_id, access_token)

        # Market-specific popularity
        markets = ['US', 'GB', 'KR']  # Add more markets as needed
        artist_1_market_popularity = get_market_specific_popularity(artist_1_id, access_token, markets)
        artist_2_market_popularity = get_market_specific_popularity(artist_2_id, access_token, markets)

        if artist_1_data and artist_2_data:
            print(f"Comparing {artist_1_name} and {artist_2_name}:\n")
            print(f"{artist_1_name} Followers: {artist_1_data['followers']['total']:,}")
            print(f"{artist_2_name} Followers: {artist_2_data['followers']['total']:,}\n")

            print(f"{artist_1_name} Popularity: {artist_1_data['popularity']}")
            print(f"{artist_2_name} Popularity: {artist_2_data['popularity']}\n")

            print(f"{artist_1_name} Genres: {', '.join(artist_1_data['genres'])}")
            print(f"{artist_2_name} Genres: {', '.join(artist_2_data['genres'])}\n")

            print(f"{artist_1_name} Top Tracks:")
            for track in artist_1_tracks:
                print(f"  - {track['name']} (Popularity: {track['popularity']})")

            print(f"\n{artist_2_name} Top Tracks:")
            for track in artist_2_tracks:
                print(f"  - {track['name']} (Popularity: {track['popularity']})")

            print(f"\n{artist_1_name} Albums:")
            for album in artist_1_albums:
                print(f"  - {album['name']}")

            print(f"\n{artist_2_name} Albums:")
            for album in artist_2_albums:
                print(f"  - {album['name']}")

            print(f"\n{artist_1_name} Playlist Appearances:")
            for playlist in artist_1_playlists:
                print(f"  - {playlist['name']}")

            print(f"\n{artist_2_name} Playlist Appearances:")
            for playlist in artist_2_playlists:
                print(f"  - {playlist['name']}")

            print(f"\n{artist_1_name} Market-Specific Popularity:")
            for market, popularity in artist_1_market_popularity.items():
                print(f"  - {market}: {popularity:.2f}")

            print(f"\n{artist_2_name} Market-Specific Popularity:")
            for market, popularity in artist_2_market_popularity.items():
                print(f"  - {market}: {popularity:.2f}")

        else:
            print("Error fetching artist data.")
    else:
        print("Error retrieving artist IDs.")

In [99]:
  
artist_1 = 'BTS'
artist_2 = 'Taylor Swift'


In [100]:
compare_artists(artist_1, artist_2, access_token)

Comparing BTS and Taylor Swift:

BTS Followers: 73,391,231
Taylor Swift Followers: 112,195,477

BTS Popularity: 84
Taylor Swift Popularity: 100

BTS Genres: k-pop, k-pop boy group, pop
Taylor Swift Genres: pop

BTS Top Tracks:
  - Dynamite (Popularity: 75)
  - My Universe (Popularity: 71)
  - Left and Right (Feat. Jung Kook of BTS) (Popularity: 72)
  - FAKE LOVE (Popularity: 72)
  - Butter (Popularity: 71)
  - Boy With Luv (feat. Halsey) (Popularity: 71)
  - Run BTS (Popularity: 72)
  - Life Goes On (Popularity: 70)
  - Spring Day (Popularity: 69)
  - Take Two (Popularity: 69)

Taylor Swift Top Tracks:
  - Fortnight (feat. Post Malone) (Popularity: 90)
  - I Can Do It With a Broken Heart (Popularity: 85)
  - Cruel Summer (Popularity: 89)
  - Down Bad (Popularity: 82)
  - Who’s Afraid of Little Old Me? (Popularity: 79)
  - Guilty as Sin? (Popularity: 79)
  - So Long, London (Popularity: 78)
  - My Boy Only Breaks His Favorite Toys (Popularity: 77)
  - But Daddy I Love Him (Popularity: 7